In [5]:
from pprint import pprint

from fish import REPO_PATH, log
from fish.data import DataLoader, Dataset

**Approach**
1. Create dataloader and fetch video,label pairs
2. Create list of (boxes, track, etc.) for each frame
3. Draw boxes on each frame
4. Write video to file as gif (standard name/location)
5. Display raw, filter, etc side by side in markdown
6. Calculate and plot metric

In [6]:
dataloader = DataLoader(
    Dataset(
        videos=f'{REPO_PATH}/data/mp4',
        labels=f'{REPO_PATH}/data/labels' 
    )
)

[INFO] 07/06/2022 08:16:52AM: MainProcess: data.py - Initializing Dataset...
[INFO] 07/06/2022 08:16:52AM: MainProcess: data.py - 23 video files found


In [15]:
video, label = next(dataloader)
print(type(video))
print(video.shape)

<class 'numpy.ndarray'>
(160, 1792, 1032, 3)


In [19]:
per_frame_boxes = [None]*video.shape[0] # pre init size?
for track in label['tracks']:
    for frame in track['frames']:
        if frame['frame'] <= label['video_length']:
            per_frame_boxes[frame['frame']] = frame.update({'label': track['label'], 'track_id': track['track_id']})
        else:
            log.warning(f"Video {label['video_id']}, track {track['track_id']} contains frame number larger than video:\nVideo length: {label['video_length']}\tFrame number: {frame['frame']}")

pprint(per_frame_boxes[150])

[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 250
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 251
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 252
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 253
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 254
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 1801226884.py - Video 13, track 135 contains frame number larger than video:
Video length: 160	Frame number: 255
[WARNING] 07/06/2022 08:34:04AM: MainProcess: 

Display gifs using Markdown

<img src="../experiments/dft/outputs/all_ac_video/demo_raw_video.gif" alt="raw_video" height="500"/>
